In [ ]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4

def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          



def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        print(sentences)
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

In [170]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('../lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

In [10]:
f = open('../../Latin/Suetonius/opensource/suet.caes_lat.xml')
ltext = f.read()
f.close()
f = open('../../Latin/Suetonius/opensource/suet.thomson_eng.xml')
etext = f.read()
f.close()

lat_sections = re.split(r'<div1 type="life" n=".*">', ltext)[1:]

eng_sections = re.split(r'<div1 type="life" n=".*">', etext)[2:]

In [30]:
sects = list(zip(lat_sections,eng_sections)) # ignores all the extra bios in the english text

In [67]:
lsects = []
esects = []

for sect in sects:
    lsects.append(re.split(r'<div2 type="chapter" n="\d+">', sect[0])[1:])
    esects.append(re.split(r'<div2 type="chapter" n="\d+">', sect[1])[1:])

In [81]:
for i in range(len(lsects)):
    print(len(lsects[i]),len(esects[i]))

89 89
101 99
76 76
60 60
46 46
57 57
23 23
12 12
18 18
25 25
11 11
23 23


In [91]:
eng_sects = esects
lat_sects = lsects

In [159]:
lat_text = []
eng_text = []

for s in range(len(eng_sects)):
    if len(eng_sects[s]) != len(lat_sects[s]):
        continue
        
    for c in range(len(eng_sects[s])):
        es = eng_sects[s][c]
        if re.match(r'</.>',es):
            es = re.split(r'</.>',es,maxsplit=1)[1]
        soup = BeautifulSoup(es,'lxml')
        if soup.find('note'):
            es = re.sub(r'<note[^>.]*>', '<note>', es,flags=re.DOTALL)
            sp = re.split(r'<note>',es)
            es = sp[0]
            for n in sp[1:]:
                n = re.sub(r'.*</note>', " ", n,flags=re.DOTALL)
                es += n
            soup = BeautifulSoup(es,'lxml')
            eng_text.append(soup.text)
        else:
            eng_text.append(soup.text)

        ls = lat_sects[s][c]
        if re.match(r'</.>',ls):
            el = re.split(r'</.>',ls,maxsplit=1)[1]
        soup = BeautifulSoup(ls,'lxml')
        if soup.find('note'):
            ls = re.sub(r'<note[^>.]*>', '<note>', ls,flags=re.DOTALL)
            sp = re.split(r'<note>',ls)
            ls = sp[0]
            for n in sp[1:]:
                n = re.sub(r'.*</note>', " ", n,flags=re.DOTALL)
                ls += n
            soup = BeautifulSoup(ls,'lxml')
            lat_text.append(soup.text)
        else:
            lat_text.append(soup.text)

In [157]:
e = re.sub(r'<note[^>.]*>', '<note>', eng_sects[0][1],flags=re.DOTALL)
sp = re.split(r'<note>',e)
t = sp[0]
for n in sp[1:]:
    n = re.sub(r'.*</note>', " ", n,flags=re.DOTALL)
    t+=(n)

In [161]:
lat_text[1]

'\nStipendia prima in Asia fecit Marci Thermi praetoris contubernio; a quo ad accersendam classem in\nBithyniam missus desedit apud Nicomeden, non sine\nrumore prostratae regi pudicitiae; quem rumorem auxit\nintra paucos rursus dies repetita Bithynia per causam\nexigendae pecuniae, quae deberetur cuidam libertino\nclienti suo. reliqua militia secundiore fama fuit et a\nThermo in expugnatione Mytilenarum corona ciuica donatus est.\n'

In [169]:
lat_text[1]

'\nStipendia prima in Asia fecit Marci Thermi praetoris contubernio; a quo ad accersendam classem in\nBithyniam missus desedit apud Nicomeden, non sine\nrumore prostratae regi pudicitiae; quem rumorem auxit\nintra paucos rursus dies repetita Bithynia per causam\nexigendae pecuniae, quae deberetur cuidam libertino\nclienti suo. reliqua militia secundiore fama fuit et a\nThermo in expugnatione Mytilenarum corona ciuica donatus est.\n'

In [171]:
alignments = []

for c in range(len(eng_text)):
    es = ' '.join(eng_text[c].replace('\n',' ').split())
    ls = lat_text[c].replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    es = re.sub(r'\. \. \.','',es)
    ls = re.sub('[:;,]','',ls)
    es = re.sub('[:;,]','',es)

    epar = sent_tokenize(es)
    lpar = s_tokenizer.tokenize_sentences(ls)
    alignments.append(get_sec_alignments(lpar,epar))

error, []
error, []


In [178]:
alignments[-1] = alignments[-1][:3]

In [193]:
alignments[201]

[(' Nepotatus sumptibus omnium prodigorum ingenia superauit commentus nouum balnearum usum portentosissima genera ciborum atque cenarum ut calidis frigidisque unguentis lauaretur pretiosissima margarita aceto liquefacta sorberet conuiuis ex auro panes et obsonia apponeret aut frugi hominem esse oportere dictitans aut Caesarem.',
  ['In the devices of his profuse expenditure he surpassed all the prodigals that ever lived inventing a new kind of bath with strange dishes and suppers washing in precious unguents both warm and cold drinking pearls of immense value dissolved in vinegar and serving up for his guests loaves and other victuals modelled in gold often saying " that a man ought either to be a good economist or an emperor."']),
 ('quin et nummos non mediocris summae e fastigio basilicae Iuliae per aliquot dies sparsit in plebem.',
  ['Besides he scattered money to a prodigious amount among the people from the top of the Julian Basilica during several days successively.']),
 ('fabri

In [182]:
len(alignments)

440

In [187]:
count = 0
for a in alignments:
        count += len(a)

In [188]:
count

2180

In [195]:
f = open('../aligned_sentences/suetonius_sentences.json','w') 
j = json.dumps(alignments)
f.write(j)
f.close()